# Работа с видео в OpenCV

**Видеоданные:** свойства матриц (тензоров) и временных рядов (есть порядок, последовательность). Это "временной ряд матриц (тензоров)", N+1-мерный пространственно-временной тензор.

Основные свойства видеоданных:

1. Число кадров в секунду (framerate). Стандартный поток - 25 fps (frames per second), т.е. каждый кадр держится на экране 40 мс


2. Разрешение видео. Наиболее популярные разрешения имеют названия по размерам **w x h**

- <b>SD</b> (Standard Definition), 640x360 or 720x480
- <b>HD</b> (High Definition), 1280x720 or 720p
- <b>Full HD</b>, 1920x1080 or 1080p
- <b>UHD</b> (Ultra HD, 4K), 3840x2160 

3. Форматы (т.н. "кодеки" - codecs) для чтенияи записи видео: AVI (давний) и MP4 (довольно новый). Определяют расширение файла. AVI не поддерживает самые современные HEVC/H.265 или VP9 форматы. HEVC и VP9 разработаны как UHD видео-кодеки. MP4 может поддерживать 4K UHD, тогда как AVI - только HD и FHD. Кроме того, AVI не поддреживает субтитров, хот может содержать объекты типа SubRip, SubStation Alpha и XSUB (в рамках сторонних модификаций).

Как и в случае **`cv2.imread`**, когда мы читаем изображение, функция **`cv2.VideoCapture`** создает объект [**VideoCapture**](https://docs.opencv.org/4.1.0/d8/dfe/classcv_1_1VideoCapture.html#ac4107fb146a762454a8a87715d9b7c96), читающий видео из локального файла или другого истчника:

``` python
VideoCapture object	= cv.VideoCapture(filename[, apiPreference])
```
Параметры:
- **`filename`** может быть:
    - имя файла (video.avi)
    - последовательность изорбражений с одинаковыми размерами (img_00.jpg, img_01.jpg, img_02.jpg, ...)
    - URL видеопотока (protocol://host:port/script_name?script_params|auth). ВРпазные видеопотоки или IP-камераы имеют разные URL-схемы (см. документацию).
- **`apiPreference`**:	предпочтительный обработчик видеопотока, обычно выьирается из нескольких существующих: cv2.CAP_FFMPEG, cv2.CAP_IMAGES, cv2.CAP_DSHOW

**Наиболее употребляемые способы** (по источникам) для чтения видео объектом VideoCapture Object:
1. Вебкамера (аргумент 0). Если подключено 2 и более камер - аргумент = порядковый номер камеры: 0, 1, 2...
2. Видеофайл (аргумент = имя файла)
3. Последовательность изображений (аргумент вида image_%03d.jpg)

## Читаем первый кадр из видео

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os # библиотека для работы с файловой системой
import shutil # еще одна библиотека для работы с файловой системой
%matplotlib inline

In [ ]:
import matplotlib # настройки matplotlib
matplotlib.rcParams['figure.figsize'] = (10.0, 10.0)
matplotlib.rcParams['image.cmap'] = 'gray'

### Файл для работы

In [ ]:
cap = cv2.VideoCapture('z2_1min.mp4') # создаем объект VideoCapture для чтения локального файла

In [ ]:
# Проверка успешного захвата видео
if not cap.isOpened(): # cap.isOpened() - свойство объекта VideoCapture, возвращаующе True или False
    print("Error opening video stream or file")
else:
     print("OK!")

Для покадрового чтения видеопотока используется функцуия **`cap.read`**, которая возвращает кортеж из двух значений: логического (True = кадр прочитан) и самого прочитанного кадра: 

In [ ]:
ret, frame = cap.read() # потому мы подставлем два имени для возвращаемых функцией значений

После прочтения видео можно вывести его на экран покадрово. Т.к. каждый кадр это изображение - можно использовать функцию **`cv2.imshow()`**. Обычно jupyter-ноутбуки не приспособлены для воспроизводства видео, но видео в них вполне возможно вставлять и воспроизводить двумя способами:
- matplotlib imshow в самом ноутбуке
- cv2.imshow в локальном python-скрипте

In [ ]:
print(type(frame))
frame.shape # объект frame - массив  numpy, обладающий большинством свойств таких массивов

In [ ]:
plt.imshow(frame) # выводим прочитанный (первый) кадр и... о ужас!

Можно также при чтении изображения или кадра указать конвертацию BGR --> RGB **`cv2.COLOR_BGR2RGB`**, поскольку BGR (по неведомым причинам) исходно родной формат для OpenCV.

In [ ]:
 # меняем  каналы местами массива (нотация среза: читаем с конца), трюк равносилен 
plt.imshow(frame[...,::-1])

## Отображение кадров в виде видео (только локальные файлы)
Как и в случае с изображением, используем функцию **`cv2.waitKey()`** после функции **`cv2.imshow()`**, чтобы приостановить каждый кадр видео. В случае изображения мы передаем **0** в функцию **`waitKey`**, но для воспроизведения видео нам нужно передать в функцию **`waitKey()`** число > **0**. Значение **0** приостанавливает кадр в видео на бесконечное время, а в видео нам нужно, чтобы каждый кадр отображался небольшое время. И нужно передать в функцию **`waitKey`** число больше **0** (время в мс, в течение которого кадр отображается).

- **`waitKey`** для веб-камеры. При чтении с веб-камеры использование **`waitKey(1)`** уместно (частота кадров дисплея будет ограничена частотой кадров веб-камеры, даже если указать задержку в 1 мс.
- **`waitKey`** для видеофайла. При чтении кадров из видео, может быть полезным установить задержку в 1-2 мс, чтобы поток был освобожден для выполнения необходимой нам обработки. В редких случаях (воспроизведение с определенной частотой кадров) задержка может быть > 1 мс.

In [ ]:
# "бесконечный" цикл с ключевым словом while: по этому слову создаются циклы, длящиеся по умолчанию (до наступления условия)
while(cap.isOpened()):
    ret, frame = cap.read() # пока кадр открывается объектом VideoCapture
    
    if ret == True: # если кадр прочитан успешно
        cv2.imshow("Video Output", frame) # вывод кадра на экран   
        cv2.waitKey(25) # ждем 25 мс для вывода следующего кадра
    
    # выход из цикла
    else: 
        break
        cap.release() # не забываем в конце освободить поток (иначе придется всякий раз перезапускать jupyter)

## Как узнать свойства видеопотка?
За это отвечают метод **`cap.get(propId)`**. Здесь **`cap`** это объект **`VideoCapture`** свойства которого надо узнать, **`propId`** это [Property ID](https://docs.opencv.org/4.1.0/d4/d15/group__videoio__flags__base.html#gaeb8dd9c89c10a5c63c139bf7c4f5704d) и **`value`** идентификатор свойства **`propId`**.

Наиболее частоиспользуемые свойства и их ID (цифры):

| Enumerator | Numerical Value | Property |
| --- | --- | --- |
| **`cv2.CAP_PROP_POS_MSEC`** | `0` | текущая позиция чтения потока в мс |
| **`cv2.CAP_PROP_FRAME_WIDTH`** | `3` | ширина кадра |
| **`cv2.CAP_PROP_FRAME_HEIGHT`** | `4` | высота кадра |
| **`cv2.CAP_PROP_FPS`** | `5` | частота кадров, к / сек |
| **`cv2.CAP_PROP_FOURCC`** | `6` | четырехбуквенное обозначение кодека обработки |


In [ ]:
width = cap.get(3) # ширина кадра
height = cap.get(4) # высота кадра

print(width,height) # для использования переводим в целочисмленные значения с помощью int()

## Как установить свойства видеопотка?
Мы можем использовать метод и **`cap.set(propId,value)`** для изменения свойств видеопотока. Например, изменить высоту и ширину входного видео во время чтения.

**Примечание**: функция set может дать неожиданные результаты (настройка свойства может быть не предусмотрена файлом или камерой). Для веб-камеры невозможно установить произвольные значения ширины и высоты кадра (не все разрешения поддерживаются). Например, разрешение камеры по умолчанию 720x1280. Если установить его на 200x200, этого может не произойти, будет установлено другое значение 640x480, которое поддерживается.

In [ ]:
# True = свойство изменено удачно, False - изменения не произошли или не соответсвуют точно заданным
ret = cap.set(3, 320) # пытаемся установить ширину
print(ret)

ret = cap.set(4, 180) # пытаемся установить высоту
print(ret)

## Запись видео в OpenCV¶
После завершения покадровой обработки видео, следующий шаг — это сохранить результат. С изображениями все просто. Нам просто нужно использовать **`cv2.imwrite()`** и укажите формат изображения (jpg/png). Для видео требуется дополнительная информация.

1. Создайте объект VideoWriter:
```python
VideoWriter object = cv.VideoWriter(filename, fourcc, fps, frameSize[, isColor])
```
**Параметры VideoWriter:**
- **`filename`**: имя выходного видеофайла.
- **`fourcc`**: 4-значный код кодека, используемый для сжатия кадров. Например, VideoWriter::fourcc('P','I','M','1') — это кодек MPEG-1, VideoWriter::fourcc('M','J','P','G ') — кодек motion-jpeg и т. д. Список кодов можно получить на странице [FourCC](https://en.wikipedia.org/wiki/FourCC). Обработчик FFMPEG с контейнером MP4 изначально использует другие значения может прийти предупреждающее сообщение (не ошибка!) от OpenCV о преобразовании кода fourcc.
- **`fps`**: частота кадров создаваемого видеопотока.
- **`frameSize`**: размер видеокадров - кортеж из пары чисел **(W, H)**.
- **`isColor`**: если не 0, обработчик будет ожидать и кодировать цветные кадры, иначе будет работать с кадрами в оттенках серого (флаг поддерживается только в Windows).

In [ ]:
cap = cv2.VideoCapture('z2_1min.mp4')

In [ ]:
frame_width = int(cap.get(3)) # получаем данные исходного (читаемого) потока
frame_height = int(cap.get(4)) # а именно - ширину и высоту кадра
codec = cv2.VideoWriter_fourcc(*'XVID') # Если файл не записался, пробуйте другие кодеки ('mp4v' или 'M','J','P','G')

# Определяем кодек и создаем объект VideoWriter. Записанное видео будет иметь имя 'z2_1min_10fps.mp4' 
# ВНИМАНИЕ! Указание имени оригинала перезапишет оригинал!
writer = cv2.VideoWriter('z2_1min_5fps.mp4', codec, 5, (frame_width,frame_height))

In [ ]:
# Читаем в условно-бесконечном цикле while (т.е. пока есть новые кадры в потоке)
while(cap.isOpened()):
    ret, frame = cap.read() # захват кадра за кадром
    
    if ret == True: # если кадр успешно открыт
        writer.write(frame) # пишем его в последовательность файла контейнера "z2_1min_5fps.mp4"
        cv2.waitKey(25) # ждем 25 мс (время на обработку)
    else: 
        break
        
cap.release() # освободждаем не только читаемы оригинал,
writer.release() # но и записываемый файл-контейнер
print('recording complete')

#### Итог: записана копия файла с  пониженной до 5 fps частотй кадров

##  Функции обработки видео
### 1. Разбор видео на кадры

In [ ]:
def video2frames(src, out, sample, xr, yr, x, y, w, h):
    
    '''
    Sample frames from video with constant frequency i 
    (write each i-th frame, parameter 'sample')
    '''
    
    if not os.path.exists(out):
        os.mkdir(out)
    
    cap = cv2.VideoCapture(src)
    
    if not cap.isOpened(): 
        print('Error opening video')
    
    i, s = 0, 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret == True:
            if i % sample == 0: # пишет каждый 'sample-th' кадр
                frame = cv2.resize(frame, (xr, yr), cv2.INTER_NEAREST)
#                 frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE) # cv2.ROTATE_90_COUNTERCLOCKWISE, cv2.ROTATE_180
#                 cv2.rectangle(frame , (y, x), (y+h, x+w), (255, 255, 255), thickness=2)
#                 frame = frame[x:w, y:h]
#                 ...
#                 ...и другие преобразования (функции OpenCV...)
                cv2.imwrite(out + '/' + os.path.basename(path)[:-4] + '_' + str(i) + '.jpg', frame)

                s += 1
            i += 1
        else:
            break
    cap.release()
        
    return f'OK, {s} frames were saved'

In [ ]:
%%time
path = 'z2_1min_5fps.mp4' # путь к видео
out = os.path.dirname(path) + 'z2_1min_5fps_frames' # путь расположения результата
cap = cv2.VideoCapture(path) # захват видео объектом VideoCapture по указанному пути
new_x, new_y = int(cap.get(3)), int(cap.get(4)) # получаем ширину и высоту кадра от оригинала, передаем в writer

# video2frames(path, out, 1, new_x, new_y, 0, 0, new_x, new_y)

### 2. Копирование подвыборки кадров в другую директорию

In [ ]:
def frame_select(src, out, sample):
    
    """
    Здесь должно быть исчерпывающее описание функции, но как обычно времени нет на это :)
    """
    if not os.path.exists(out):
        os.mkdir(out)
    
    names = os.listdir(src)
    i, n = 0, 0
    for name in names:
        if i % sample == 0:
            shutil.copy2(src + '/' + name, out + '/' + name)
            n += 1
        i += 1
    
    return f'{n} frames were selected'

In [ ]:
path = 'z2_1min_5fps_frames'
src = os.path.dirname(path) + 'z2_1min_5fps_frames'
out = os.path.dirname(path) + 'z2_1min_5fps_frames_selected'

frame_select(src, out, 2) # итог: каждый второй кадр перенесен в другую диреторию

### Сбор видео из кадров

In [ ]:
def frames2video(src, out, rate, w, h):
    
    '''
    Тут должна быть документация по функции, но как обычно нет времени :)
    '''
       
    fourcc = cv2.VideoWriter_fourcc(*'XVID') #*'mp4v'
    wrt = cv2.VideoWriter(out, fourcc, rate, (w, h))
    names = os.listdir(os.getcwd() + src)
    
    for name in names:
        frame = cv2.imread(os.getcwd() +  src + name)
#         frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)
        wrt.write(frame)
        
    wrt.release()
    
    return 'OK'

In [ ]:
%%time
path = 'z2_1min_5fps_frames_selected'
src = os.path.dirname(path) + '\z2_1min_5fps_frames_selected'
out = os.path.dirname(path) + '\z2_1min_5fps_frames_selected.mp4'

frames2video(src, out, 25, 720, 1280)

## FFmpeg
Блиотека-основа, которую использует "под капотом" OpenCV. Типичный конвеер обработки файа в FFmpeg: сперва указывается последостельность обработок в виде команд (функций с параметрами), после чего давется комнада на выполнение данной цепи преоразований.

In [ ]:
import ffmpeg

In [ ]:
# полная информация о файле
stream = ffmpeg.probe('z2_1min.mp4')
stream